Set up 

In [ ]:
"""Byte pair encoding utilities"""

import os
import json
import regex as re
from functools import lru_cache

import tensorflow as tf 


In [ ]:
# sử dụng @lru_cache để lưu trữ kết quả của 1 hàm vào bộ nhớ cake 
# đồng thời giúp tăng tốc độ trương trình 
@lru_cache 
# Xây dựng phương thức trợ giúp để tạo một từ điển ánh xạ từ các byte UTF8 sang các ký tự unicode
def bytes_to_unicode():
    """
    Trả về danh sách của byte utf8 và một danh sách của chuỗi unicode tương ứng 
    Mã bpe có thể đảo ngược hoạt động trên chuỗi unicode . 
    Điều này có nghĩa cần nhiều ký tự unicode trong tập từ vựng nếu muốn tránh UNKS.

    """
    # Tạo một danh sách bs chứa các mã số nguyên của các kỹ tự từ ! đến ~ , từ i đến "¬" .. 
    # các ký tự này là các ký tự thông dụng trong bảng ASCII và Latin-1 . Hàm ord trả trảveef mã số 
    # nguyên của một ký tự . Hàm Range trả về 1 số từ một số bắt đầu đêbs 1 số kết thúc 
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    # sao chép kết quả list bs gán cho biến cs 
    cs = bs[:]
    # khởi tạo 1 biến đếm n  = 0
    n = 0
    # lặp qua 1 dnah sách từ 0 -> 256 tương ứng với giá trị byte có thể
    for b in range(2**8):
        # kiểm tra xem giá trị byte b có trong danh sách bs 
        if b is not bs: 
            # nếu không ta thêm b vào danh sách bs 
            bs.append(b)
            # đồng thời thêm vào cuối danh sách cs 1 giá trị = 256 + n . Số này là mã số nguyên
            # của một ký tự Unicode mới không trùng với các ký tự trong danh sách bs 
            cs.append(2**8 + n)
            # tăng n lên 1 
            n += 1
    
    # Tạo một danh sách mới cs bằng cách chuyển đổi các số trong danh sách cũ thành các ký tự Unicode
    # Hàm chr trả về 1 ký tự tương ứng với 1 mã số nguyên 
    cs = [chr(n) for n in cs]
    # sử dụng hàm zip để kết hợp hai danh sách bs và cs theo cặp tương ứng 
    #( ký tự đặc biệt , ký tự thay thế) hàm dict tạo một từ điển từ các cặp này 
    # trả về bảng tra cứu cuối cùng 
    return dict(zip(bs, cs))
        

# Thiết lập phương thức xử lý ghép cặp các ký tự liền kề 
def get_pair(word):
    """Trả về một tập hợp các cặp ký tự trong một từ 
        Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    # gán biến pairs là kiểu set để chánh phải tồn tại các cặp ký tự trùng lặp trong danh sách 
    pairs = set()
    # gán gái trị prev_char = ký tự đầu tiền của từ 
    prev_char = word[0]
    # duyệt qua các ký tự trong từ , tính từ chỉ số ký tự thứ 2
    for char in word[1:]:
        # thêm vào danh sách pairs các cặp từ theo chỉ số vị trí tăng 
        pairs.add((prev_char, char))
        # sau đó gán lại biến prev_char là kết quả của char để có thể có 1 cặp khác 
        prev_char = char 
    return pairs 




Encoding 

In [ ]:
class Encoder: 
    # Thiết lập phương thức khởi tạo và định nghĩa các tham số 
    def __init__(self, encoder , bpe_merges, errors="replace"):
        self.encoder = encoder 
        # Tạo một bộ giải mã từ bộ mã hóa , đảo ngược các khóa và giá trị . 
        # với bộ mã hóa encoder là một từ điển ánh xạ từ các ký tự Unicode sang mã số 
        # nguyên . Bộ giải mã thì ngược lại
        self.decoder = {v:k for k,v in self.encoder.items()}
        # gán các giá trị cho thuộc tính self.errors chỉ định cách xử lý các lỗi 
        # trong quá trình giải mã. có thể là replace, ignore hoặc strict 
        self.errors = errors 
        # gọi hàm bytes_to_unicode để tạo một từ điển ánh xạ các byte UTF-8 sang các ký tự Unicode 
        # hàm này trả về 1 từ điển có 256 khóa tương đương với 256 byte có thể có trong UTF-8. 
        # giá trị mỗi khóa là một ký tự unicode duy nhất không trùng lặp với các ký tự thông thường 
        self.byte_encoder = bytes_to_unicode()
        # Tạo 1 từ điển ánh xạ từ các cặp byte được ghép nối trong BPE sang các thứ hạng của chúng
        # bpe_merges là một danh sách các cặp byte được sắp xếp theo thứ tự ưu tiên của chúng 
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
        # Tạo một từ điển rỗng để lưu trữ kết quả của các phép mã hóa BPE đã thực hiện 
        # để tránh lặp phép tính 
        self.cache = {}

        # Tạo một đối tượng biểu thức chính quy và gán cho thuọco tính self.pat 
        # biểu thức chính quy này tiềm kiếm có thể là các từ rút gọn trong chuỗi.. 
        # Should haved added re.IGNORECASE so BPE merges can happen for capitalized versions of contractions
        self.pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

    # Thiết lập phương thức mã hóa cặp byte pairs encoding 
    # nhận tham số token là chuỗi cần mã hóa 
    def bpe(self, token):
        # kiểm tra xem chuỗi token có nằm trong từ điển cache tức là chuỗi này đã được xử lý trước đó 
        if token in self.cache:
            # thì trả về giá trị tương ứng của nó đã được mã hóa trước đó 
            return self.cache[token]
        # nếu chuỗi token không nằm trong từ điển cache lưu nó vào tuple nhằm 
        # cho các giá trị trong chuỗi không thể thay đổi 
        word = tuple(token)
        # sau đó ta thực hiện mã háo cặp cho chuỗi đó sử dụng phương thức get_pairs(word)
        pairs = get_pair(word)

        # kiểm tra xem tập hợp pairs có rỗng hay không 
        if not pairs:
            # nếu rỗng nghĩa là bộ word chỉ có một phần tử hoặc không có phần từ nào tức nguyên vẹn  
            return token 

        # Tạo một vòng lặp While True vô hạn đến khi Break 
        while True: 
            # Tìm cặp byte có thứ hạng thấp nhất trong tập hợp pairs 
            # Tham số key là một tham số để so sánh các phần tử 
            # hàm lambda pair tar về thứ hạng của các cặp bpe trong từ điển self.bpe_ranks 
            # nêu cặp không có trong từ điển thì trả về một số nguyên lớn 
            bigram = min(pairs , key = lambda pair: self.bpe_ranks.get(pair , float('intf')))
            # kiểm tra xem cặp byte có trong từ điển bpe_rank hay không 
            if bigram not in self.bpe_ranks: 
                # Kết thúc vòng lặp 
                break 

            # Gán giá trị của first và second cho cặp phần tử byte 
            first , second = bigram 
            # Tạo một danh scahs dỗng để lưu trữ kết quả của quá trình mã hóa BPE . 
            # Một danh sách là một cấu trúc dữ liệu có thứ tự có thể thay đổi , có thể chứa 
            # cacds phần tử khác nhau 
            new_word = []
            i = 0 # biến i sẽ được sử dụng để duyệt qua các phần tử trong bộ từ 
            # tạo một vòng lặp cho tới khi giá trị của biến i có độ dài = word 
            while i < len (word):
                # Khởi tạo một khối xử lý ngoại lệ có thể xảy ra trong quá trình thực thi các lệnh 
                try : 
                    # tìm vị trí đầu tiên của phần tử first trong bộ word bắt đầu tại vị trí i 
                    # và gán nó cho biến j 
                    j = word.index(first , i)
                    # Thêm các phần tử từ vị trí i đến vị trí j vào danh scahs neword 
                    new_word.extend(word[i:j])
                    # Gán vị trí hiện tại của j cho i để cập nhật vị trí hiện tại của biến i 
                    # trong bộ word 
                    i = j 
                # bắt đầu 1 khối expect để xử lý các ngoại lệ được ném ra từ khối try 
                except: 
                    # Thêm các phần tử từ vị trí i đến cuối bộ word vào danh sách word_neư
                    new_word.extend(word[i:])
                    # kết thúc vòng lặp 
                    break # có nghĩa là không tìm thấy phần tử first trong bộ word 

                # kiểm tra xem 3 điều kiện sau đây có đều đúng hay không 
                # chỉ số i trong từ điển word có bằng với first , vị trí i nhỏ hơn độ dài bộ word trừ 1 
                # và phần tử tại vị trí i + 1 tính từ first có bằng second 
                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    # Thêm 1 chuỗi first + second vào danh sách new_word là kết qủa của phép nối 
                    # hai ký tự liền kề trong bộ word
                    new_word.append(first + second)
                    # tăng giá trị của i lên 2 . Tức là cập nhật vị trí hiện tại của biến i trong bộ 
                    # word để ở lần thực thi tiếp theo bỏ qua 2 phần tử đã được ghép nối trước đó 
                    i += 2 
                
                # Trường hợp còn lại thêm vào từ điển new_word phần tử theo vị trí i của từ điểm word 
                else : 
                    new_word.append(word[i])
                    # Tăng giá trị của biến i lên 1 
                    i += 1

            # gán lại kết qủa của new_word  = 1tuple để các giá trị của nó không thay đổi 
            new_word = tuple(new_word)
            # gán kết quả mới cho word = new_word 
            word = new_word
            # kiểm tra xem độ đài danh sách word = 1 
            if len(word) == 1 :
                # thì dừng lại 
                break 
            else: 
                # Gán danh sách pairs = kết quả của việc mã hóa ghép cặp các từ tronh danh sách word 
                pairs = get_pair(word)
        
        # Nối các phần tử trong danh sách word ngăn cách nhau bằng 1 khoảng trắng 
        word = ' '.join(word)
        # Lưu trữ vào bộ nhớ cache một danh sách word các bộ từ đã được xử lý tương ứng với 
        # chuỗi token 
        self.cache[token] = word 

        # sau đó trả về danh sách word 
        return word 
    
    # Thiết lập phương thức encode nhận vào là 1 chuỗi văn bản và trả về một danh sách cá số nguyên 
    # là chỉ số của token BPE trong bộ từ vựng . Phương thức này sử dụng 1 biểu thức cính quy 
    # để tách văn bản thành các toke , một bộ byte_encoder để chuyển đổi các byte trong token 
    # thành các ký tự Unicode tương ứng . và một bộ encoder để ánh xạ các token BPE thành các chỉ số.
    def encode(self, text):
        # khởi tạo 1 danh sách rỗng để lưu trữ kết quả
        bpe_tokens = []
        # sử dụng biểu thức chính quy self,pat để tách văn bản thành các token
        # hàm re.findall có chức năng để tìm tất cả các kết quả khớp vơí biểu thức chính quy 
        for token in re.findall(self.pat , text): 
            # với mỗi token trong chuỗi văn bản chuyển nó thành các ký tự Unicode 
            # bằng cách sử dụng byte_encoder . sau đó sử dụng self.bpe để chuyển đổi 
            # chuỗi này thành 1 chuỗi các token BPE 
            token = ''.join(self.byte_encoder[b] for b in token.encode('utf-8'))
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' '))

        # Trả về danh sách bpe_tokens. Đây là danh sách các số nguyên, biểu diễn mã hóa BPE của văn bản đầu vào.
        return bpe_tokens 
    
    # Thiết lập phương giải mã xử lý quy trình ánh xạ ngược một dnah sách các số nguyên 
    # là chỉ số của token BPE trong bộ từ vựng và trả về 1 chuỗi văn bản 
    def decode(self, tokens):
        # Với mỗi số nguyên trong danh sách tokens tìm token BPE tương ứng bằng cách sử dụng 
        # bộ decoder 
        text = ''.join([self.decoder[token] for token in tokens])
        # Chuyển đổi chuỗi text thành một mảng byte bằng cách sử dụng bộ byte_encoder
        # mỗi ký tự trong chuỗi text sẽ được chuyển đổi thành byte tương ứng 

        # Giải mã byte thành một chuỗi văn bản bằng cách sử dụng phuwong thức decode của lớp 
        # bytearray , tham số utf8 chỉ định chuỗi mã hóa của byte 
        text = bytearray([self.byte_decoder[c] for c in text]).decode('utf-8', errors=self.errors)
        return text



In [ ]:
# thiết lập phuwong thức tạo ra một đối tượng Encoder từ một đường dẫn chưa các tệp encoder json
# và vocab.bpe
# Đối tượng Encoder là một lớp để mã hóa và giải mã văn bản theo phương pháp BPE (Byte Pair Encoding).
# Nhận vào một tham số encoder_path, là đường dẫn đến thư mục chứa các tệp tin encoder.json và vocab.bpe. 
# Tệp tin encoder.json là một tệp tin định dạng JSON, chứa một từ điển ánh xạ các token BPE thành các chỉ số.
# Tệp tin vocab.bpe là một tệp tin văn bản, chứa một danh sách các cặp byte được ghép nối 
# theo thứ tự tần suất xuất hiện giảm dần trong tập dữ liệu huấn luyện
def get_encoder(encoder_path):
    
    # Sử dụng phương thức tf.gfile.Open để mở tệp tin encoder.json ở chế độ đọc (‘r’). 
    # Phương thức này trả về một đối tượng file, được gán cho biến f.
    # Sử dụng từ khóa with để đảm bảo đóng tệp tin sau khi thực hiện xong các lệnh trong khối.
    with tf.gfile.Open(os.path.join(encoder_path, 'encoder.json'), 'r') as f:
        # Sử dụng hàm json.load để đọc nội dung của tệp tin encoder.json và chuyển đổi nó thành một từ điển Python, được gán cho biến encoder. 
        encoder = json.load(f)
    
    # Sử dụng phương thức tf.gfile.Open để mở tệp tin vocab.bpe ở chế độ đọc (‘r’). Phương thức này trả về một đối tượng file, được gán cho biến f.
    with tf.gfile.Open(os.path.join(encoder_path, 'vocab.bpe'), 'r') as f: # utf-8?
        # Sử dụng phương thức f.read để đọc toàn bộ nội dung của tệp tin vocab.bpe và trả về một chuỗi, được gán cho biến bpe_data.
        bpe_data = f.read()

    # Tạo một danh sách bpe_merges rỗng để lưu trữ các cặp byte được ghép nối. Sử dụng một biểu thức danh sách (list comprehension) 
    # để duyệt qua các dòng trong chuỗi bpe_data, bỏ qua dòng đầu tiên và dòng cuối cùng
        
    # Mỗi dòng trong chuỗi bpe_data được tách ra bằng ký tự xuống dòng (‘\n’). 
    # Với mỗi dòng, sử dụng phương thức split để tách dòng thành hai phần tử, là hai byte được ghép nối.
    # Sau đó, sử dụng hàm tuple để chuyển đổi hai phần tử này thành một bộ (tuple), 
    # và thêm bộ này vào danh sách bpe_merges.
    
    # vd  nếu chuỗi bpe_data là:#version: 0.2
    # e s
    # s t
    # t a
    # a r
    # r t
    # thì danh sách bpe_merges sẽ là: [('e', 's'), ('s', 't'), ('t', 'a'), ('a', 'r'), ('r', 't')]

    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]

    # Trả về đối tượng Encoder. Đây là kết quả của hàm get_encoder.
    return Encoder(
        encoder=encoder,
        bpe_merges=bpe_merges,
)     